In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv("http://datatraining.site/data/GLE-PA-data.csv")

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Close']])

# Create the training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Prepare the data for LSTM
def create_dataset(dataset, time_steps=1):
    X, y = [], []
    for i in range(len(dataset) - time_steps - 1):
        a = dataset[i:(i + time_steps)]
        X.append(a)
        y.append(dataset[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 30
X_train, y_train = create_dataset(train_data, time_steps)
X_test, y_test = create_dataset(test_data, time_steps)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')

# Make predictions and generate trading signals
predictions = model.predict(X_test)
signals = []
for i in range(len(predictions)):
    if predictions[i] > y_test[i]:
        signals.append(1)  # Buy signal
    else:
        signals.append(-1)  # Sell signal

# Calculate the trading strategy performance
initial_capital = 10000
portfolio_value = [initial_capital]
for i in range(1, len(signals)):
    if signals[i] == 1:
        portfolio_value.append(portfolio_value[-1] + (test_data[i] - test_data[i-1]) * portfolio_value[-1])
    elif signals[i] == -1:
        portfolio_value.append(portfolio_value[-1] - (test_data[i] - test_data[i-1]) * portfolio_value[-1])
    else:
        portfolio_value.append(portfolio_value[-1])

print(f'Final Portfolio Value: ${portfolio_value[-1]}')

Epoch 1/50
12/12 [==============================] - 4s 43ms/step - loss: 0.1089
Epoch 2/50
12/12 [==============================] - 1s 47ms/step - loss: 0.0258
Epoch 3/50
12/12 [==============================] - 1s 46ms/step - loss: 0.0171
Epoch 4/50
12/12 [==============================] - 1s 47ms/step - loss: 0.0179
Epoch 5/50
12/12 [==============================] - 1s 50ms/step - loss: 0.0157
Epoch 6/50
12/12 [==============================] - 1s 67ms/step - loss: 0.0150
Epoch 7/50
12/12 [==============================] - 0s 34ms/step - loss: 0.0136
Epoch 8/50
12/12 [==============================] - 0s 28ms/step - loss: 0.0137
Epoch 9/50
12/12 [==============================] - 0s 29ms/step - loss: 0.0122
Epoch 10/50
12/12 [==============================] - 0s 29ms/step - loss: 0.0144
Epoch 11/50
12/12 [==============================] - 0s 29ms/step - loss: 0.0124
Epoch 12/50
12/12 [==============================] - 0s 28ms/step - loss: 0.0110
Epoch 13/50
12/12 [==================